In [ ]:
!pip install --upgrade pandavro agcounts

In [ ]:
import pandas as pd
import pandavro as pdx
from agcounts.extract import get_counts

In [ ]:
!wget https://github.com/actigraph/icampam_2024_workshop/raw/main/input/big.avro

In [ ]:
def acc_from_avro(avro_file: str, timezone: str) -> pd.DataFrame:
    raw_acc = pdx.read_avro(avro_file)
    frequency = 1 / raw_acc.SampleOrder.max()
    raw_acc.Timestamp = raw_acc.Timestamp + raw_acc.SampleOrder * frequency
    raw_acc.Timestamp = pd.to_datetime(raw_acc.Timestamp, unit="s")
    raw_acc.set_index("Timestamp", inplace=True)
    raw_acc.tz_localize(timezone)
    return raw_acc


def counts_from_df(raw_acc: pd.DataFrame, epoch: int) -> pd.DataFrame:
    counts = get_counts(raw_acc[["X", "Y", "Z"]].values, freq=32, epoch=epoch)
    indexes = (
        raw_acc.index.to_series()
        .groupby(pd.Grouper(freq=f"{epoch}s", origin="start_day"))
        .first()
    )
    counts = pd.DataFrame(counts, columns=["X", "Y", "Z"], index=indexes[: len(counts)])
    return counts


In [ ]:
def magnitude(counts: pd.DataFrame) -> pd.Series:
    # TODO
    pass


def apply_cutpoint(magnitude_counts: pd.Series, cutpoint: int) -> pd.Series:
    # TODO
    pass


def daily(data: pd.Series) -> pd.Series:
    # TODO
    pass


In [ ]:
raw_data = acc_from_avro(
    "big.avro", "US/Central"
)

In [ ]:
counts_60 = counts_from_df(raw_data, 60)
magnitude_counts = magnitude(counts_60)
intensity = apply_cutpoint(magnitude_counts, 3000)
days = daily(intensity)
print(days)

In [ ]:
def counts_to_csv(counts: pd.DataFrame, filename: str):
    counts.to_csv(filename)

def counts_from_csv(filename: str, epoch: int) -> pd.DataFrame:
    counts_1s = pd.read_csv(filename, parse_dates=[0], index_col="Timestamp")

    # TODO Calculate counts for epoch
    return counts

In [ ]:
counts_1s = counts_from_df(raw_data, 1)
counts_to_csv(counts_1s, "counts_1s.csv")
new_counts_60 = counts_from_csv("counts_1s.csv", 60)
magnitude_counts = magnitude(new_counts_60)
intensity = apply_cutpoint(magnitude_counts, 3000)
days = daily(intensity)
print(days)

In [ ]:
# Check if the two ways of obtaining counts are the same